In [17]:
import os
import src.data.data as data

IMG_SIZE = 224
BATCH_SIZE = 128
NO_OF_OUTPUT_CLASSES = 27

TRAIN_DIR = data.get_output_dir(IMG_SIZE, IMG_SIZE, True, False, "train")
VALIDATION_DIR = data.get_output_dir(IMG_SIZE, IMG_SIZE, True, False, "test")

CHECKPOINT_DIR = os.path.join("data", "models", "cnn_mobilenetv2")
CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, "cp_{epoch:04d}.ckpt")

print(CHECKPOINT_PATH)

data\models\cnn_mobilenetv2\cp_{epoch:04d}.ckpt


In [18]:
import tensorflow_hub as hub
from tensorflow.keras import layers, Sequential, regularizers

# Download mobilenet model from tensorflow hub
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
feature_extractor = hub.KerasLayer(URL,
                                   input_shape=(IMG_SIZE, IMG_SIZE,3),
                                   trainable=True)

model = Sequential([
    layers.InputLayer(input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    feature_extractor,
    layers.Dropout(rate=0.2),
    layers.Dense(NO_OF_OUTPUT_CLASSES,
                 kernel_regularizer=regularizers.l2(0.0001))
])
model.build((None, IMG_SIZE, IMG_SIZE, 3))

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


image_gen_train = ImageDataGenerator(
      rescale=1./255,               # Normalization
      # rotation_range=40,
      # width_shift_range=0.2,                
      # height_shift_range=0.2,
      # shear_range=0.2,
      # zoom_range=0.2,
      # horizontal_flip=True,
      # fill_mode='nearest'
)

image_gen_validation = ImageDataGenerator(
    rescale=1./255, # Normalization
)

train_data_gen = image_gen_train.flow_from_directory(batch_size=BATCH_SIZE,     # Batch size means at a time it takes 100
                                                     directory=TRAIN_DIR,    # Here we put shuffle= True so tat model doesnt memorise order
                                                     shuffle=True,
                                                     target_size=(IMG_SIZE,IMG_SIZE),
                                                     class_mode='categorical')

validation_data_gen = image_gen_validation.flow_from_directory(batch_size=BATCH_SIZE,     # Batch size means at a time it takes 100
                                                     directory=VALIDATION_DIR,    # Here we put shuffle= True so tat model doesnt memorise order
                                                     shuffle=True,
                                                     target_size=(IMG_SIZE,IMG_SIZE),
                                                     class_mode='categorical')


Found 67932 images belonging to 27 classes.
Found 16984 images belonging to 27 classes.


In [20]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.train import latest_checkpoint

model.compile(
  optimizer=SGD(learning_rate=0.005, momentum=0.9), 
  loss=CategoricalCrossentropy(from_logits=True, label_smoothing=0.1),
  metrics=['accuracy'])


latest = latest_checkpoint(CHECKPOINT_PATH)
if(latest is not None):
    print("Loading checkpoint", latest)
    model.load_weights(latest)
else:
    print("No checkpoint to load")

# This callback will stop the training when there is no improvement in
# the accuracy for three consecutive epochs.
cp_callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=3),
    ModelCheckpoint(CHECKPOINT_PATH,
                    save_weights_only=True,
                    verbose=1)
    ]

steps_per_epoch = train_data_gen.samples // BATCH_SIZE
validation_steps = validation_data_gen.samples // BATCH_SIZE
hist = model.fit(
    train_data_gen,
    epochs=100, steps_per_epoch=steps_per_epoch,
    validation_data=validation_data_gen,
    validation_steps=validation_steps,
    callbacks=cp_callbacks).history

No checkpoint to load
Epoch 1/100


ResourceExhaustedError: Graph execution error:

Detected at node 'train/MobilenetV2/Conv_1/Conv2D' defined at (most recent call last):
Node: 'train/MobilenetV2/Conv_1/Conv2D'
OOM when allocating tensor with shape[1280,320,1,1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node train/MobilenetV2/Conv_1/Conv2D}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_70401]